In [139]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [140]:
using JuMP, GLPK
using DataFrames
using CSV

In [141]:
#load demand data
demand = CSV.File("../data/demand.csv") |> DataFrame

#load cost data
cost = CSV.File("../data/cost_generation.csv") |> DataFrame

#load capacity data
capacity = CSV.File("../data/technical_data_generation.csv") |> DataFrame

#load wind_technicaldata
wind = CSV.File("../data/wind_technicaldata.csv") |> DataFrame


Row,Unit,Node,Pmax (MW),Pmin (MW),Ci ($/MWh)
,Int64,Int64,Int64,Int64,Int64
1,1,3,200,0,0
2,2,5,200,0,0
3,3,7,200,0,0
4,4,16,200,0,0
5,5,21,200,0,0
6,6,23,200,0,0


In [142]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
 G = size(cost, 1)

 # number of demand
 D = size(demand_bids, 1)

 # number of wind generators
 W = size(wind, 1)

6

In [143]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], objective_value = Float64[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

In [144]:

for hour in 1:24
  # Create a new model with GLPK solver
  model = Model(GLPK.Optimizer)

  # Define the decision variables for every generator
  unregister(model, :x)  # Unregister the existing variable named "x" from the model
  
  #demand bids
  name = "demand_hour_" * string(hour-1) * ".csv"
  demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame

   # number of demand
   D = size(demand_bids, 1)

  # decision variables for convential generators
  @variable(model, x[1:G])  # Create the new variable
  for generator in 1:G
      set_lower_bound(x[generator], 0)  # Set the lower bound
      set_upper_bound(x[generator], capacity[generator,3])  # Set the upper bound
  end

  # wind decision variables
  @variable(model, x_wind[1:W])  # Create the new variable
  for generator in 1:W
      set_lower_bound(x_wind[generator], wind[generator,4])  # Set the lower bound
      set_upper_bound(x_wind[generator], wind[generator,3])  # Set the upper bound
  end

  # decision variables for demand
  @variable(model, y[1:size(demand_bids, 1)])  # Create the new variable
  for bid in 1:D
      set_lower_bound(y[bid], 0)  # Set the lower bound
      set_upper_bound(y[bid], demand_bids[bid,2])  # Set the upper bound
  end


  # Define the objective function
  @objective(model, Max, sum(demand_bids[d,3] * y[d] for d in 1:D) - sum(cost[g,2] * x[g] for g in 1:G) - sum(wind[w,5] * x_wind[w] for w in 1:W))

  # Add the constraint for the balance between supply and demand
  @constraint(model, sum(x[g] for g in 1:G) + sum(x_wind[w] for w in 1:W) - sum(y[d] for d in 1:D) == 0)

  # Print the model
  #print(model)

  # Solve the model
  optimize!(model)

  # Check the status of the solution
  status = termination_status(model)
  if status == MOI.OPTIMAL
      println("Hour ", hour, ": Optimal solution found")
      #println("Objective value: ", objective_value(model))
      #println("x values: ", [value(x[i]) for i in 1:G])
      #println("x_wind values: ", [value(x_wind[i]) for i in 1:W])
      #println("y values: ", [value(y[i]) for i in 1:D])

      #save results to dataframe
      push!(result_df, [hour, objective_value(model), [value(x[i]) for i in 1:G]..., [value(x_wind[i]) for i in 1:W]..., [value(y[i]) for i in 1:D]...])

  elseif status == MOI.INFEASIBLE
      println("Hour ", hour, ": No feasible solution found")
  else
      println("Hour ", hour, ": Solver status: ", status)
  end
end

Hour 1: Optimal solution found
Hour 2: Optimal solution found
Hour 3: Optimal solution found
Hour 4: Optimal solution found
Hour 5: Optimal solution found
Hour 6: Optimal solution found
Hour 7: Optimal solution found
Hour 8: Optimal solution found
Hour 9: Optimal solution found
Hour 10: Optimal solution found
Hour 11: Optimal solution found
Hour 12: Optimal solution found
Hour 13: Optimal solution found
Hour 14: Optimal solution found
Hour 15: Optimal solution found
Hour 16: Optimal solution found
Hour 17: Optimal solution found
Hour 18: Optimal solution found
Hour 19: Optimal solution found
Hour 20: Optimal solution found
Hour 21: Optimal solution found
Hour 22: Optimal solution found
Hour 23: Optimal solution found
Hour 24: Optimal solution found


In [145]:
result_df

Row,hour,objective_value,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x_wind1,x_wind2,x_wind3,x_wind4,x_wind5,x_wind6,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,41809.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,0.0,0.0,95.2252,32.9464,162.903,180.231,0.0,225.391,141.869,237.728,0.0,75.0261,0.0,113.932,97.8283,250.871,211.581,196.878,166.503,111.087
2,2,33939.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0174,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,167.498,0.0,174.502,0.0,225.063,0.0,0.0,242.232,0.0,271.104,0.0,107.957,90.0358,248.875,114.826,0.295911,72.2837,200.341,14.0036,0.0
3,3,36129.4,0.0,0.0,0.0,0.0,0.0,155.0,155.0,400.0,400.0,300.0,310.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,0.0,0.0,205.748,183.773,285.208,70.5899,0.0,106.515,230.358,186.891,0.0,257.544,177.87,239.477,283.08,234.743,147.964,64.3073,0.0,245.932
4,4,42496.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0767,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,182.63,102.793,0.0,62.6313,41.3109,56.1774,55.5472,0.0,0.0,289.055,212.036,64.7428,59.8237,250.882,283.437,0.0,6.32088,0.0,229.531,59.1585
5,5,36938.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,174.173,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,215.837,39.8994,0.0,212.611,0.0,62.5487,249.087,0.0,296.192,115.12,201.519,44.1636,0.0,94.536,287.188,0.0787097,0.0,154.256,0.0,101.138
6,6,40653.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,49.7495,250.305,128.252,214.502,24.4921,13.9228,72.5966,0.0,181.272,257.832,152.179,298.377,125.375,5.98868,254.488,0.0,62.2707,159.903,36.0081,12.485
7,7,56435.9,0.0,0.0,0.0,0.0,0.0,155.0,72.7641,400.0,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,0.0,185.813,102.653,59.4073,7.82298,179.922,288.642,176.784,224.531,200.23,173.568,192.86,113.591,0.0,261.646,204.581,152.209,0.0,3.50342,0.0
8,8,38532.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,183.727,83.8165,44.804,0.0,272.878,0.0,0.0,99.5922,0.0,152.481,109.201,9.59198,260.706,279.702,11.2783,226.811,0.0,86.5742,52.2719,26.5641
9,9,37581.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.547,400.0,300.0,0.0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,0.0,79.1139,11.1967,0.0,124.032,40.1477,253.186,291.307,73.6302,49.6137,193.48,0.0,0.0,141.179,150.099,87.8987,195.298,0.0,107.321,211.046


In [146]:
#save result_df to csv
CSV.write("../results/market_clearing_Jan.csv", result_df)

"../results/market_clearing_Jan.csv"